In [ ]:
pip install gym

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install gymnasium

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 41.1 MB/s eta 0:00:00


In [ ]:
pip install Ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
import numpy as np
import gymnasium as gym
import random
import time
from IPython.display import clear_output

In [41]:
# Create the environment
env = gym.make('FrozenLake-v1', render_mode='ansi')


In [42]:
# Initialize the Q-table
# A Q-Table helps us to find the best action for each state in the environment.
state_space_size = env.observation_space.n # number of rows in q table
action_space_size = env.action_space.n # number of columns in q table

q_table = np.zeros((state_space_size, action_space_size)) #building and filling the q table with zeros
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [43]:
# Set hyperparameters
num_episodes = 10000
max_steps_per_episode = 100 # if agent havent reach the goal by 100 step agent will recieve 0 points and new episode will start


learning_rate = 0.1 # aplha
discount_rate = 0.99 #gamma

# epsilon greedy strategy
exploration_rate = 1 #epsilon
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001 # determine the rate at which the exploration rate will decay
# first agent explore the environment epsilon = 1 then after each episode epsilon start to decrease and probability of
# exploring decrease and tend toward exploiting the environment.

In [44]:
# List of rewards
rewards_all_episodes = []

# 2 Q learning algorithm - For life or until learning is stopped
for episode in range(num_episodes): #this loop contain everything that happen with a single episode
    # Reset the environment back to the starting state
    state = env.reset()[0]
    done = False # starting out with environment state not done
    rewards_current_episode = 0 # starting out with zero rewards

    for step in range(max_steps_per_episode): #this loop contain everything that happen with a single time step in each episode
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        ### Exploration - exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1) #this will be used to determine if agen will explore or exploit the environment

        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) # chossing the biggest q value

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, truncated, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        q_table[state, action] = q_table[state, action] * (1-learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))

        # new q value is the weighted sum of old q value and the learned value
        # Learning rate in how quickly the agent abendends the q value in Q-table for new q-value



        # Our new state is state
        state = new_state
        rewards_current_episode += reward

        # If done (if we're dead) : finish episode
        if done == True:
            break

    # Reduce epsilon (because we need less and less exploration)
    ## Exploration rate decay
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)

    rewards_all_episodes.append(rewards_current_episode)


# calculate and print the average rewards per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000
print("*****Average reward per thousand episodes******\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000
# print updated q table
print("\n\n*****Q-table*****")
print(q_table)

*****Average reward per thousand episodes******

1000 :  0.05000000000000004
2000 :  0.22500000000000017
3000 :  0.4160000000000003
4000 :  0.5490000000000004
5000 :  0.6590000000000005
6000 :  0.6590000000000005
7000 :  0.6560000000000005
8000 :  0.6850000000000005
9000 :  0.6990000000000005
10000 :  0.6500000000000005


*****Q-table*****
[[0.53322358 0.47235546 0.45467521 0.45637953]
 [0.33219048 0.3597539  0.41038779 0.45439638]
 [0.38524148 0.38636832 0.38504659 0.42948074]
 [0.27601341 0.33285804 0.28375168 0.39901958]
 [0.55537446 0.30543983 0.34491143 0.41334572]
 [0.         0.         0.         0.        ]
 [0.14118134 0.14672884 0.32006351 0.12740045]
 [0.         0.         0.         0.        ]
 [0.31591014 0.3670882  0.40306568 0.59584287]
 [0.40402617 0.6752164  0.40744566 0.35121178]
 [0.56232421 0.34513962 0.31672077 0.33036058]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.52665916 0.49187804 0.77264255 0.40995431]


In [45]:
# Watch our agent play Frozen Lake by playing the best action
# from each state according to the Q-table

for episode in range(5):
    # initialize new episode params
    state = env.reset()[0]
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)

    for step in range(max_steps_per_episode):
        # Show current state of environment on screen
        # Choose action with highest Q-value for current state
        # Take new action
        clear_output(wait=True) # clear the output for new episode
        print(env.render()) # to see the game grid and see where our agent is
        time.sleep(0.3)

        action = np.argmax(q_table[state,:])
        new_state, reward, done, truncated, info= env.step(action)


        if done:
            clear_output(wait=True)
            print(env.render())
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
                clear_output(wait=True)
            break

        state = new_state


env.close()

  (Down)
SFFF
FHFH
FFFH
HFFG

****You reached the goal!****
